In [1]:
import numpy as np
import itertools
from scipy.stats import poisson

In [2]:
location_a=[i for i in range(0,21)]
location_b=[i for i in range(0,21)]
States = list(itertools.product(location_a, location_b))

In [3]:
#if action 5 is taken location a gets 5 cars that night, if -5 is taken location b gets 5 cars.
action_space = (5,4,3,2,1,0,-1,-2,-3,-4,-5)
#max_a stands for maximum number of vehicles can be transferred to location a 
actions={}

for x in range(len(States)):
    
    max_a=min(5,States[x][1],20-States[x][0])
    min_a=-min(States[x][0],5,20-States[x][1])
    actions[States[x]]=[i for i in range(min_a,max_a+1)]


In [4]:
def poisson_distribution(k, lambd):
    return (lambd ** k * np.exp(-lambd)) / np.math.factorial(k)

In [5]:
transition_probs = {}
rewards = {}
cars_rent=(0,0)

for s in States:
    for a in actions[s]:
        for s2 in States:

            cars_available_a=s[0]+a
            cars_available_b=s[1]-a
  
            cars_maxreturned_a=s2[0] 
            cars_maxreturned_b=s2[1] 

            rent_options_a=[i for i in range(cars_available_a-min(cars_available_a,cars_maxreturned_a), cars_available_a+1)]
            rent_options_b=[i for i in range(cars_available_b-min(cars_available_b,cars_maxreturned_b), cars_available_b+1)]
            
            loc_a_prob=1
            loc_b_prob=1
            
            
            # these if statements fills the probability of having requests more than existing cars, 
            # and returns more than location capability.
            # Calculated using 1-negative probability. Might have been used mass function instead of for.
            
            rewards_a=[]
            rewards_b=[]
            
            for i in range(len(rent_options_a)):
                
                negative_prob_rent=0
                negative_prob_return=0

                rewards_a.append(rent_options_a[i]*10)

                if  rent_options_a[i]==s[0]+a:
                    
                    for z in range(rent_options_a[i]):
                        negative_prob_rent=negative_prob_rent+poisson_distribution(z,3)
                        
                            
                    if s2[0]!=20:                                
                        loc_a_prob=loc_a_prob*(1-negative_prob_rent)*poisson_distribution(s2[0]-(s[0]+a-rent_options_a[i]),3)
                        
                        
                    else:

                        for p in range(20):
                               negative_prob_return=negative_prob_return+poisson_distribution(p,3)
                                
                        loc_a_prob=loc_a_prob*(1-negative_prob_rent)*(1-negative_prob_return) 
    

                else:
                    if s2[0]!=20:
                        loc_a_prob=loc_a_prob*poisson_distribution(rent_options_a[i],3)*poisson_distribution(s2[0]-(s[0]+a-rent_options_a[i]),3)
                    else:
                        for p in range(20):
                               negative_prob_return=negative_prob_return+poisson_distribution(p,3)
                                
                        loc_a_prob=loc_a_prob*poisson_distribution(rent_options_a[i],3)*(1-negative_prob_return)      

                        
            for i in range(len(rent_options_b)):
    
                negative_prob_rent_b=0
                negative_prob_return_b=0
                
                rewards_b.append(rent_options_b[i]*10)
                
                if  rent_options_b[i]==s[1]-a:
                    
                    for z in range(rent_options_b[i]):
                        negative_prob_rent_b=negative_prob_rent_b+poisson_distribution(z,4)
                            
                    if s2[1]!=20:
                                
                        loc_b_prob=loc_b_prob*(1-negative_prob_rent_b)*poisson_distribution(s2[1]-(s[1]-a-rent_options_b[i]),2)
                        
                    else:

                        for p in range(20):
                               negative_prob_return_b=negative_prob_return_b+poisson_distribution(p,2)
                                
                        loc_b_prob=loc_b_prob*(1-negative_prob_rent_b)*(1-negative_prob_return_b)  
                        
                else:
                    if s2[1]!=20:

                        loc_b_prob=loc_b_prob*poisson_distribution(rent_options_b[i],4)*poisson_distribution(s2[1]-(s[1]-a-rent_options_b[i]),2)

                    else:
                        for p in range(20):
                               negative_prob_return_b=negative_prob_return_b+poisson_distribution(p,2)
                                                                
                        loc_b_prob=loc_b_prob*poisson_distribution(rent_options_b[i],4)*(1-negative_prob_return_b)  
                        

            transition_probs[s,a,s2]=loc_a_prob*loc_b_prob
            
            rewards[(s, a, s2)]=-abs(a)*2
            
            for i in range(len(rewards_a)):
                rewards[(s, a, s2)]=rewards[(s, a, s2)]+rewards_a[i]
                
            for i in range(len(rewards_b)):
                rewards[(s, a, s2)]=rewards[(s, a, s2)]+rewards_b[i]

In [37]:
def policy_evaluation(policy):
    V = {}
    for s in States:
        V[s] = 0
    boundary = 1e-4 
    gamma = 0.9
    
    it = 0
    while True:
        biggest_change = 0
        for s in States:
            old_v=V[s]
            new_v = 0 
            for a in action_space:
                for s2 in States:
                
                    if policy.get(s)==a:
                        action_prob=1
                    else:
                        action_prob=0
                    
                    rew = rewards.get((s, a, s2),0)
                    new_v += action_prob*transition_probs.get((s, a, s2),0) * (rew + gamma * V[s2])
            
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        print("iter:", it, "biggest_change:", biggest_change)
        it += 1
        if biggest_change < boundary:
            break
    print("\n\n")
    return(V)

In [38]:
policy={}
for s in actions.keys():
    policy[s] = np.random.choice(actions.get(s))


In [39]:
while True:
    gamma = 0.9
    
    V = policy_evaluation(policy)
    
    is_policy_converged = True
    
    for s in States:
        
        old_a = policy[s]
        new_a = None
        best_value = float('-inf')
        
        for a in actions.get(s):
            v = 0
            for s2 in States:
            
                r = rewards.get((s, a, s2), 0)
                v += transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])
            if v > best_value:
                best_value = v
                new_a = a
              
                policy[s] = new_a
        if new_a != old_a:
            is_policy_converged = False
    if is_policy_converged:
        break

iter: 0 biggest_change: 3.3656038540905198
iter: 1 biggest_change: 0.3019767135064256
iter: 2 biggest_change: 0.0051378955197561305
iter: 3 biggest_change: 9.57905162355388e-05



iter: 0 biggest_change: 3.3656172222784075
iter: 1 biggest_change: 1.785710424763261
iter: 2 biggest_change: 0.0815711540335875
iter: 3 biggest_change: 0.003715907011098585
iter: 4 biggest_change: 0.00015774735655704752
iter: 5 biggest_change: 6.417319196927451e-06



iter: 0 biggest_change: 3.3656172222784075
iter: 1 biggest_change: 1.785710424763261
iter: 2 biggest_change: 0.0815711540335875
iter: 3 biggest_change: 0.003715907011098585
iter: 4 biggest_change: 0.00015774735655726957
iter: 5 biggest_change: 6.417319196927451e-06





In [40]:
policy

{(0, 0): 0,
 (0, 1): 0,
 (0, 2): 0,
 (0, 3): 0,
 (0, 4): 0,
 (0, 5): 0,
 (0, 6): 0,
 (0, 7): 0,
 (0, 8): 0,
 (0, 9): 0,
 (0, 10): 0,
 (0, 11): 0,
 (0, 12): 5,
 (0, 13): 5,
 (0, 14): 5,
 (0, 15): 5,
 (0, 16): 5,
 (0, 17): 5,
 (0, 18): 5,
 (0, 19): 5,
 (0, 20): 5,
 (1, 0): -1,
 (1, 1): -1,
 (1, 2): -1,
 (1, 3): -1,
 (1, 4): -1,
 (1, 5): -1,
 (1, 6): -1,
 (1, 7): -1,
 (1, 8): -1,
 (1, 9): -1,
 (1, 10): -1,
 (1, 11): 4,
 (1, 12): 4,
 (1, 13): 5,
 (1, 14): 5,
 (1, 15): 5,
 (1, 16): 5,
 (1, 17): 5,
 (1, 18): 5,
 (1, 19): 5,
 (1, 20): 5,
 (2, 0): -2,
 (2, 1): -2,
 (2, 2): -2,
 (2, 3): -2,
 (2, 4): -2,
 (2, 5): -2,
 (2, 6): -2,
 (2, 7): -2,
 (2, 8): -2,
 (2, 9): -2,
 (2, 10): 3,
 (2, 11): 3,
 (2, 12): 4,
 (2, 13): 4,
 (2, 14): 5,
 (2, 15): 5,
 (2, 16): 5,
 (2, 17): 5,
 (2, 18): 5,
 (2, 19): 5,
 (2, 20): 5,
 (3, 0): -3,
 (3, 1): -3,
 (3, 2): -3,
 (3, 3): -3,
 (3, 4): -3,
 (3, 5): -3,
 (3, 6): -3,
 (3, 7): -3,
 (3, 8): -3,
 (3, 9): 2,
 (3, 10): 2,
 (3, 11): 3,
 (3, 12): 3,
 (3, 13): 4,
 (3, 14):